[View in Colaboratory](https://colab.research.google.com/github/kundajelab/ssvmimp/blob/master/lsgkm.ipynb)

In [1]:
#install lsgkm
! git clone https://github.com/Dongwon-Lee/lsgkm
% cd lsgkm/src
! make
%cd ../..

Cloning into 'lsgkm'...
remote: Enumerating objects: 96, done.
remote: Total 96 (delta 0), reused 0 (delta 0), pack-reused 96
Unpacking objects: 100% (96/96), done.
/content/lsgkm/src
g++ -Wall -Wconversion -O3 -fPIC -c libsvm.cpp
g++ -Wall -Wconversion -O3 -fPIC -c libsvm_gkm.c
g++ -Wall -Wconversion -O3 -fPIC gkmtrain.c libsvm.o libsvm_gkm.o -o gkmtrain -lm -lpthread
g++ -Wall -Wconversion -O3 -fPIC gkmpredict.c libsvm.o libsvm_gkm.o -o gkmpredict -lm -lpthread
/content


In [2]:
#get the raw data
!wget https://raw.githubusercontent.com/AvantiShri/model_storage/master/deeplift/genomics/sequences.simdata.gz


Redirecting output to ‘wget-log’.


In [3]:
!zcat sequences.simdata | head

seqName	sequence	embeddings	task1	task2	task3
empty_1732	AGCTCTTTTGGCGGACCGAGTTCGTCGGACTTTCGCTAAATTGACCATTAACAGGAGGATACCCGGGTGTCTCGGGAAGGTCGTAATACGGTAATTACGGATGTGAGAGCGCACACATCCTTTGGATGATACATGATCTCGCCGTATATGGCCTAGGTCAAAAGGACAGTCTCGCTAATCGCAATAATGCAGCTAACCGA		0	0	0
gata_only_1527	TGCAGATAAGAACTTTGGGAATATGGTAGGAAAAACCCCTGAAGTGAGATCCAAAAAATATAGACTGGCGCGGTGGCAATGCCCTTGTTAATGCTGGGAATCAAAAAATCGAAACTCTCAGCAGATAAGATATGCAGACCTGCTCGTGGAGACAAAAAGTATTGTTACTGCGTAAACCATTAGTACGTTGCCGAGTACCG	pos-120_GATA_disc1-GCAGATAAGA,pos-1_GATA_disc1-GCAGATAAGA	0	1	0
gata_tal1_62	AGATTATAGGCCCTCCACCAGATAAGATATAGCGTTCGAGGGTCAGAAGGTAGTTCTGAAGATCTCTTCATTGTGAAAGGCCTCATATGATGAGCTTCTCCATTCTCAGCTGACAGGCAAAACAATTGATCAGCGCTTAGGCGCCAGATGTTTACTCCCGTTAAGTCAACTTAGCTGTGTTCGTCAAATCCGCAACGCGG	pos-17_GATA_disc1-CCAGATAAGA,pos-139_TAL1_known1-GGCGCCAGATGTTTAC	1	1	1
empty_1289	TTTGGGCGCGGAAGAAGTCGCTCGCCTTTCTGTACTTGAAAATGATTACATGTAAGGATGTAACTCAATCGCAACATTGTCATTGGGTTCCATCGCCCTAGCGGTAGTAGGTTTTACCTCCAACTTGACAACCCTTCGTAAATATAAAATAGTAGTAA

In [0]:
!zcat sequences.simdata.gz | perl -lane 'if ($F[3] eq "1") {print ">$F[0]\n$F[1]"}' > positives.fa
!zcat sequences.simdata.gz | perl -lane 'if ($F[3] eq "0") {print ">$F[0]\n$F[1]"}' > negatives.fa
!perl -ne 'if ($.%10 == 1 || $.%10 == 2) {print $_}' positives.fa > test_positives.fa
!perl -ne 'if ($.%10 != 1 && $.%10 != 2) {print $_}' positives.fa > train_positives.fa
!perl -ne 'if ($.%10 == 1 || $.%10 == 2) {print $_}' negatives.fa > test_negatives.fa
!perl -ne 'if ($.%10 != 1 && $.%10 != 2) {print $_}' negatives.fa > train_negatives.fa

In [5]:
!wc -l *.fa

  12000 negatives.fa
   4000 positives.fa
   2400 test_negatives.fa
    800 test_positives.fa
   9600 train_negatives.fa
   3200 train_positives.fa
  32000 total


In [6]:
!lsgkm/src/gkmtrain


Usage: gkmtrain [options] <posfile> <negfile> <outprefix>

 train gkm-SVM using libSVM

Arguments:
 posfile: positive sequence file (FASTA format)
 negfile: negative sequence file (FASTA format)
 outprefix: prefix of output file(s) <outprefix>.model.txt or
            <outprefix>.cvpred.txt

Options:
 -t <0 ~ 5>   set kernel function (default: 4 wgkm)
              NOTE: RBF kernels (3 and 5) work best with -c 10 -g 2
                0 -- gapped-kmer
                1 -- estimated l-mer with full filter
                2 -- estimated l-mer with truncated filter (gkm)
                3 -- gkm + RBF (gkmrbf)
                4 -- gkm + center weighted (wgkm)
                     [weight = max(M, floor(M*exp(-ln(2)*D/H)+1))]
                5 -- gkm + center weighted + RBF (wgkmrbf)
 -l <int>     set word length, 3<=l<=12 (default: 11)
 -k <int>     set number of informative column, k<=l (default: 7)
 -d <int>     set maximum number of mismatches to consider, d<=4 (default: 3)
 -g <float>

In [25]:
import time

t=0
c=10
g=2
l=6
k=4
d=0

!lsgkm/src/gkmtrain -t $t -c $10 -g $2 -l $l -k $k -d $d -w 3 train_positives.fa train_negatives.fa "params_t"$t"_l"$l"_k"$k"_d"$d"_g"$g"_c"$c"_w3"


start = time.time()
!lsgkm/src/gkmpredict test_positives.fa "params_t"$t"_l"$l"_k"$k"_d"$d"_g"$g"_c"$c"_w3.model.txt" preds_positives.txt
end = time.time()
print("Time taken per seq:",(end-start)/400)
start = time.time()
!lsgkm/src/gkmpredict test_negatives.fa "params_t"$t"_l"$l"_k"$k"_d"$d"_g"$g"_c"$c"_w3.model.txt" preds_negatives.txt
end = time.time()
print("Time taken per seq:",(end-start)/1200)

import numpy as np
from sklearn.metrics import roc_auc_score
pos_preds = [float(x.rstrip().split("\t")[1]) for x in open("preds_positives.txt")]
neg_preds = [float(x.rstrip().split("\t")[1]) for x in open("preds_negatives.txt")]
print(roc_auc_score(y_true=np.array([1 for i in range(len(pos_preds))]+[0 for i in range(len(neg_preds))]),
                    y_score=np.array(pos_preds+neg_preds)))

INFO 2018-09-26 22:02:44: Number of threads is set to 1
INFO 2018-09-26 22:02:44: Arguments:
INFO 2018-09-26 22:02:44:   posfile = train_positives.fa
INFO 2018-09-26 22:02:44:   negfile = train_negatives.fa
INFO 2018-09-26 22:02:44:   outprefix = params_t0_l6_k4_d0_g2_c10_w3
INFO 2018-09-26 22:02:44: Parameters:
INFO 2018-09-26 22:02:44:   kernel-type = 0
INFO 2018-09-26 22:02:44:   L = 6
INFO 2018-09-26 22:02:44:   k = 4
INFO 2018-09-26 22:02:44:   d = 0
INFO 2018-09-26 22:02:44:   C = 10
INFO 2018-09-26 22:02:44:   w = 3
INFO 2018-09-26 22:02:44:   eps = 0.001
INFO 2018-09-26 22:02:44:   shrinking = no
INFO 2018-09-26 22:02:44: reading 1600 sequences from train_positives.fa
INFO 2018-09-26 22:02:44: reading... 0
INFO 2018-09-26 22:02:44: reading... 1000
INFO 2018-09-26 22:02:44: reading... done
INFO 2018-09-26 22:02:44: reading 4800 sequences from train_negatives.fa
INFO 2018-09-26 22:02:44: reading... 0
INFO 2018-09-26 22:02:44: reading... 1000
INFO 2018-09-26 22:02:45: reading... 2

In [31]:
!head -20 params_t0_l6_k4_d0_g2_c10_w3.model.txt

svm_type c_svc
kernel_type gkm_cnt
L 6
k 4
d 0
nr_class 2
total_sv 2701
rho 11.356
label 1 -1
nr_sv 881 1820
SV
16.79987553645302 TGTTTATATCTGCCTGCGATGAGTAAATTGTCTCTTCTGAGCCAGCGGCCAGATGGTCGGGAGACTTCGGGTCATAACGATGACTTTTATATGCTGGCCAGATAACCAACCGATATTAACAAGTGAGCAACGTCACCTCGAAACGTGCCTAAAGGTCCCCCAGTCTTCCAGCTAGTCTCAGGAGATAATCCGGCGGAATG
24.19400138278311 CTTGTATTGAGTTAGAAACCGCCACGGCACTGCTATGTATGACATTCTAACTAAGTGAGTTATGCGTTGGGTCCTTTATGTGGCATTATCTGGTAATACTTAATTGATGTACTATTTCCTCGACAAAACAGGTGGTGTGGGTGTACCGGTCACCGATAAGGGGGAACTCACCAGATGGTAGTTAACCTATAGAGTCCTGA
12.59096239848534 TTCTACTTCACCCAGAATACAGCCATCAGCAATCAGCGAACAGATGTTAACTTCTAATAATAGATATGACGATGTTCAACGAGTTTATCCCCCGGATAGACACTACTCAAGCGGATCTTGTTCTTAGGTAGGATGCCGTCCACAAGTGACGTGCCATGATACAGATAAGAACCTAAGACTCCCCCAAAAGATCAGAAATT
17.46003800529206 GCGACAAACAGGTGGCGGGCGGAGTCGGCTGAGCTATTCTCGTTTAGAATGCTTAGGTTGGCCGCTCAGAGTCCTAATGCCCTGATAAGGCGACTTTCAAACACGATCAGATGGTCAGCGACTCGGCATAATACCTTAGCTCCACAGAAAGTCGGCACCTAAGCGCGTCTACTATTGCGAGCTACCTGGAGCGTAAGAGG
21.499934402

In [115]:
import numpy as np
from collections import Counter
import itertools
from functools import reduce
import numpy as np


class GkmSvm(object):
  
  letter_to_num = {'A':0, 'C': 1, 'G': 2, 'T': 3}
  
  def __init__(self, kernel_type, l, k, d, gamma, rho,
               string_svs, signed_alphas):
    self.kernel_type = kernel_type
    self.l = l
    self.k = k
    self.d = d
    assert d==0, "Support for mismatches not implemented yet"
    self.gamma = gamma
    self.rho = rho
    self.string_svs = string_svs
    self.signed_alphas = np.array(signed_alphas)
    self.build()
    
  def build(self):
    self.build_nongap_pos_sets()
    self.normalized_gkmer_count_svs = [self.get_normalized_gkmer_counts(x)
                                       for x in self.string_svs]
    self.gkmer_feature_space_size = len(self.nongap_pos_sets)*(4**self.k)
    
  def build_nongap_pos_sets(self):
    #get the placement of the non-gap positions
    redundant_nongap_pos_sets = list(itertools.combinations(range(self.l),
                                                            self.k))
    non_redundant_nongap_pos_sets = []
    string_reps_non_redundant_nongap_pos_sets = []
    for nongap_pos_set in redundant_nongap_pos_sets:
      string_rep_arr = [" " for i in range(self.l)]
      for a_pos in nongap_pos_set:
        string_rep_arr[a_pos] = "X"
      string_rep = "".join(string_rep_arr)
      string_rep = string_rep.lstrip().rstrip()
      if (string_rep in string_reps_non_redundant_nongap_pos_sets):
        pass
        #print("Skipping",string_rep)
      else:
        string_reps_non_redundant_nongap_pos_sets.append(string_rep)
        non_redundant_nongap_pos_sets.append(nongap_pos_set)
    self.nongap_pos_sets = non_redundant_nongap_pos_sets 
    nongap_pos_set_stringrep_to_idx = dict([(x,i) for (i,x)
      in enumerate(string_reps_non_redundant_nongap_pos_sets)])
    nongap_pos_set_rc_mapping = {}
    for string_rep in string_reps_non_redundant_nongap_pos_sets:
      orig_idx = nongap_pos_set_stringrep_to_idx[string_rep]
      rc_idx = nongap_pos_set_stringrep_to_idx[string_rep[::-1]]
      nongap_pos_set_rc_mapping[orig_idx] = rc_idx
    self.nongap_pos_set_rc_mapping = nongap_pos_set_rc_mapping
  
  def get_normalized_gkmer_counts(self, a_string):
    number_string = self.encode_as_numbers(a_string)
    k = self.k
    nongap_pos_sets = self.nongap_pos_sets
    to_count_up = []
    for i in range(len(number_string)+1-l):
      for (nongap_pos_set_num,nongap_pos_set)\
        in enumerate(nongap_pos_sets):
        fwd_gkmer = 0
        rev_gkmer = 0
        for nongap_pos_enumidx, nongap_pos in enumerate(nongap_pos_set):
          #multiply the digit by the appropriate place value given by
          #nongap_pos_enumidx, add together
          fwd_gkmer |= number_string[i+nongap_pos] << (nongap_pos_enumidx << 1)
          rev_gkmer |= (3-number_string[i+nongap_pos]) << (
                        (self.k-1-nongap_pos_enumidx) << 1)
        #(1 << (k << 1)) is the same as 4**k
        fwd_gkmer += nongap_pos_set_num*(1 << (k << 1))
        rev_gkmer += self.nongap_pos_set_rc_mapping[nongap_pos_set_num]*(
                                                                  1 << (k << 1))
        to_count_up.append(fwd_gkmer)
        to_count_up.append(rev_gkmer)
    counter_obj = Counter(to_count_up)
    norm = np.linalg.norm(list(counter_obj.values()))
    normalized_counts = dict([(x,y/norm) for x,y in counter_obj.items()])
    return normalized_counts
  
  def compute_dot_product(self, norm_gkmer_counts1, norm_gkmer_counts2):
    product = 0
    for a_key in norm_gkmer_counts1:
      if a_key in norm_gkmer_counts2:
        product += norm_gkmer_counts1[a_key]*norm_gkmer_counts2[a_key]
    assert np.abs(product) <= 1, (product, norm_gkmer_counts1, norm_gkmer_counts2)
    return product
    
  def encode_as_numbers(self, a_string):
    return [self.letter_to_num[x] for x in a_string]
  

class GaussianGkmSvm(GkmSvm):
  
  def compute_prediction(self, a_string):
    normalized_gkmer_counts = self.get_normalized_gkmer_counts(a_string)
    svs_dot_products = []
    for normalized_gkmer_count_sv in self.normalized_gkmer_count_svs:
      svs_dot_products.append(self.compute_dot_product(
        norm_gkmer_counts1=normalized_gkmer_count_sv,
        norm_gkmer_counts2=normalized_gkmer_counts))
    svs_dot_products = np.array(svs_dot_products)
    return (np.sum(self.signed_alphas*np.exp(self.gamma*(svs_dot_products-1)))
            - self.rho)
  
  @classmethod
  def from_txt_file(cls, txt_file):
    lines = [x for x in open(txt_file)]
    
    field_name, kernel_type = lines[1].split()
    assert field_name=="kernel_type"
    assert kernel_type=="gkmrbf"
    field_name, l = lines[2].split()
    assert field_name=="L"; l = int(l)
    field_name, k = lines[3].split()
    assert field_name=="k"; k = int(k)
    field_name, d = lines[4].split()
    assert field_name=="d"; d = int(d)
    field_name, gamma = lines[5].split()
    assert field_name=="gamma"; gamma = float(gamma)
    field_name, rho = lines[8].split()
    assert field_name=="rho"; rho = float(rho)
    
    string_svs = []
    signed_alphas = []
    assert lines[11].rstrip() == "SV"
    for a_line in lines[12:]:
      the_alpha, the_string = a_line.split()
      the_alpha = float(the_alpha)
      string_svs.append(the_string)
      signed_alphas.append(the_alpha)

    return cls(kernel_type=kernel_type,
        l=l, k=k, d=d, gamma=gamma, rho=rho, string_svs=string_svs,
        signed_alphas=np.array(signed_alphas))

  
class LinearGkmSvm(GkmSvm):

  def compute_prediction(self, a_string):
    normalized_gkmer_counts = self.get_normalized_gkmer_counts(a_string)
    svs_dot_products = []
    for normalized_gkmer_count_sv in self.normalized_gkmer_count_svs:
      svs_dot_products.append(self.compute_dot_product(
        norm_gkmer_counts1=normalized_gkmer_count_sv,
        norm_gkmer_counts2=normalized_gkmer_counts))
    svs_dot_products = np.array(svs_dot_products)
    return (np.sum(self.signed_alphas*svs_dot_products) - self.rho)

  @classmethod
  def from_txt_file(cls, txt_file):
    lines = [x for x in open(txt_file)]
    
    field_name, kernel_type = lines[1].split()
    assert field_name=="kernel_type"
    assert kernel_type=="gkm_cnt"
    field_name, l = lines[2].split()
    assert field_name=="L"; l = int(l)
    field_name, k = lines[3].split()
    assert field_name=="k"; k = int(k)
    field_name, d = lines[4].split()
    assert field_name=="d"; d = int(d)
    field_name, rho = lines[7].split()
    assert field_name=="rho"; rho = float(rho)
    
    string_svs = []
    signed_alphas = []
    assert lines[10].rstrip() == "SV"
    for a_line in lines[11:]:
      the_alpha, the_string = a_line.split()
      the_alpha = float(the_alpha)
      string_svs.append(the_string)
      signed_alphas.append(the_alpha)

    return cls(kernel_type=kernel_type,
        l=l, k=k, d=d, gamma=None, rho=rho, string_svs=string_svs,
        signed_alphas=np.array(signed_alphas))



open("fake_gkmsvm_model.txt","w").write("""svm_type c_svc
kernel_type gkm_cnt
L 6
k 5
d 0
nr_class 2
total_sv 2
rho 1
label 1 -1
nr_sv 1 1
SV
1.0 ACGTGTACATTTATAGCAT
-1.0 GTGATCAAATGTCATA""")
open("fake_fasta_file.fa","w").write(""">seqAAAAAA
AAAAAA
>seqTTTTTT
TTTTTT
>seqTTATAC
TTATAC
>seqCCCCCC
CCCCCC""")


dummy_gkm_svm_model = LinearGkmSvm.from_txt_file("fake_gkmsvm_model.txt")
print(dummy_gkm_svm_model.compute_prediction("AAAAAA"))
print(dummy_gkm_svm_model.compute_prediction("TTTTTTT"))
print(dummy_gkm_svm_model.compute_prediction("TTATAC"))
print(dummy_gkm_svm_model.compute_prediction("CCCCCC"))

!lsgkm/src/gkmpredict fake_fasta_file.fa fake_gkmsvm_model.txt preds_fake_fasta.txt
!cat preds_fake_fasta.txt

-1.0
-1.0
-0.9487010823957422
-1.0
INFO 2018-09-26 23:23:29: Number of threads is set to 1
INFO 2018-09-26 23:23:29: load model fake_gkmsvm_model.txt
INFO 2018-09-26 23:23:29: write prediction result to preds_fake_fasta.txt
INFO 2018-09-26 23:23:29: 4 scored
seqAAAAAA	-1
seqTTTTTT	-1
seqTTATAC	-1
seqCCCCCC	-1


In [0]:
#This currently fails:

open("fake_gkmsvm_model.txt","w").write("""svm_type c_svc
kernel_type gkm_cnt
L 6
k 5
d 0
nr_class 2
total_sv 2
rho 1
label 1 -1
nr_sv 1 1
SV
1.0 ACGTGTACATTTATAGCAT
-1.0 GTGATCAAATGTCATA""")
open("fake_fasta_file.fa","w").write(""">seqAAAAAA
AAAAAA
>seqTTTTTT
TTTTTT
>seqTTATACACGGGTATACCAGTGGG
TTATACACGGGTATACCAGTGGG
>seqCCCCCC
CCCCCC""")


dummy_gkm_svm_model = LinearGkmSvm.from_txt_file("fake_gkmsvm_model.txt")
print(dummy_gkm_svm_model.compute_prediction("AAAAAA"))
print(dummy_gkm_svm_model.compute_prediction("TTTTTTT"))
print(dummy_gkm_svm_model.compute_prediction("TTATACACGGGTATACCAGTGGG"))
print(dummy_gkm_svm_model.compute_prediction("CCCCCC"))

!lsgkm/src/gkmpredict fake_fasta_file.fa fake_gkmsvm_model.txt preds_fake_fasta.txt
!cat preds_fake_fasta.txt

In [99]:
open("fake_gkmsvm_model.txt","w").write("""svm_type c_svc
kernel_type gkm_cnt
L 6
k 5
d 0
nr_class 2
total_sv 4
rho 1
label 1 -1
nr_sv 2 2
SV
1.0 AAAAAAA
1.0 TTTTTTT
-1.0 GGGGGGG
-1.0 CCCCCCC""")
open("fake_fasta_file.fa","w").write(""">seqAAAAAA
AAAAAA
>seqTTTTTT
TTTTTT
>seqCCCTTT
CCCTTT
>seqCCCCCC
CCCCCC""")


dummy_gkm_svm_model = LinearGkmSvm.from_txt_file("fake_gkmsvm_model.txt")
print(dummy_gkm_svm_model.compute_prediction("AAAAAA"))
print(dummy_gkm_svm_model.compute_prediction("TTTTTTT"))
print(dummy_gkm_svm_model.compute_prediction("CCCTTT"))
print(dummy_gkm_svm_model.compute_prediction("CCCCCC"))

!lsgkm/src/gkmpredict fake_fasta_file.fa fake_gkmsvm_model.txt preds_fake_fasta.txt
!cat preds_fake_fasta.txt

{0: 0}

In [104]:
model_file = "params_t0_l6_k4_d0_g2_c10_w3.model.txt"
gkm_svm_model = LinearGkmSvm.from_txt_file(model_file)

idx = 2
print(gkm_svm_model.compute_prediction(test_pos_seqs[idx]))
print(pos_preds[idx])

#scan a sequence to get all the kmers
#map kmer --> gapped (& mismatch kmers)

24.810802026261392
1.37348


In [60]:
!lsgkm/src/gkmpredict test_positives.fa $model_file preds_positives.txt
end = time.time()
print("Time taken per seq:",(end-start)/400)
start = time.time()
!lsgkm/src/gkmpredict test_negatives.fa $model_file preds_negatives.txt
end = time.time()
print("Time taken per seq:",(end-start)/1200)

import numpy as np
from sklearn.metrics import roc_auc_score
test_pos_seqs = [x.rstrip() for i,x in enumerate(open("test_positives.fa"))
                  if i%2==1]
test_neg_seqs = [x.rstrip() for i,x in enumerate(open("test_negatives.fa"))
                  if i%2==1]
pos_preds = [float(x.rstrip().split("\t")[1]) for x in open("preds_positives.txt")]
neg_preds = [float(x.rstrip().split("\t")[1]) for x in open("preds_negatives.txt")]
print(roc_auc_score(y_true=np.array([1 for i in range(len(pos_preds))]+[0 for i in range(len(neg_preds))]),
                    y_score=np.array(pos_preds+neg_preds)))

INFO 2018-09-26 22:45:04: Number of threads is set to 1
INFO 2018-09-26 22:45:04: load model params_t0_l6_k4_d0_g2_c10_w3.model.txt
INFO 2018-09-26 22:45:04: reading... 1000/2701
INFO 2018-09-26 22:45:04: reading... 2000/2701
INFO 2018-09-26 22:45:04: write prediction result to preds_positives.txt
INFO 2018-09-26 22:45:04: 100 scored
INFO 2018-09-26 22:45:04: 200 scored
INFO 2018-09-26 22:45:04: 300 scored
INFO 2018-09-26 22:45:04: 400 scored
Time taken per seq: 5.117107373476029
INFO 2018-09-26 22:45:05: Number of threads is set to 1
INFO 2018-09-26 22:45:05: load model params_t0_l6_k4_d0_g2_c10_w3.model.txt
INFO 2018-09-26 22:45:05: reading... 1000/2701
INFO 2018-09-26 22:45:05: reading... 2000/2701
INFO 2018-09-26 22:45:05: write prediction result to preds_negatives.txt
INFO 2018-09-26 22:45:05: 100 scored
INFO 2018-09-26 22:45:05: 200 scored
INFO 2018-09-26 22:45:05: 300 scored
INFO 2018-09-26 22:45:05: 400 scored
INFO 2018-09-26 22:45:05: 500 scored
INFO 2018-09-26 22:45:05: 600 s

63.42162434136639
1.37348


INFO 2018-09-26 22:48:22: Number of threads is set to 1
INFO 2018-09-26 22:48:22: load model fake_gkmsvm_model.txt
INFO 2018-09-26 22:48:22: write prediction result to preds_fake_fasta.txt
INFO 2018-09-26 22:48:22: 3 scored
seqAAAAAA	1
seqTTTTTT	1
seqCCCCCC	-1


0.4999999999999999

In [71]:
dummy_gkm_svm_model.normalized_gkmer_count_svs

[{0: 0.7071067811865475, 16380: 0.7071067811865475},
 {2730: 0.7071067811865475, 5460: 0.7071067811865475}]

In [12]:
import time
s = time.time()
gkm_svm_model.get_normalized_gkmer_counts(gkm_svm_model.string_svs[0])
print(time.time()-s)

0.004787921905517578


In [13]:
gkm_svm_model.gkmer_feature_space_size

2560

In [14]:
from functools import reduce
reduction_func = lambda x,y: x|y

def interpret_in_base_4(arr):
  return reduce(reduction_func,
                (y<<(x<<1) for (x,y) in enumerate(arr[::-1])))

bah = [3,3,2,0,3,2,1,2,1,2]

import time

start = time.time()
for i in range(10000):
  eh = bah[0] << 18| bah[1] << 16 | bah[2] << 14 | bah[3] << 12 | bah[4] << 10 | bah[5] << 8 | bah[6] << 6 | bah[7] << 4 | bah[8] << 2 | bah[9]
print(time.time()-start)

start = time.time()
for i in range(10000):
  eh = interpret_in_base_4(bah)
print(time.time()-start)


print(eh)
print(interpret_in_base_4(bah))

0.009868383407592773
0.04675936698913574
1019494
1019494


In [0]:
from collections import Counter
c = Counter()

In [0]:
c.update("a")

In [17]:
c

Counter({'a': 1})

In [18]:
oink |= 2
oink

NameError: ignored

In [1]:
#Let's time how long ISM would take

!wc -l test_positives.fa


wc: test_positives.fa: No such file or directory


In [59]:
!head preds_negatives.txt

empty_1732	-0.926003
tal1_only_1375	-0.719885
tal1_only_932	-0.643103
tal1_only_822	-0.311129
gata_only_1864	-0.764027
tal1_only_258	-0.170276
tal1_only_479	-0.423172
gata_only_253	-0.725754
gata_only_838	-0.581721
gata_only_1803	-0.921533


0.8409916666666667
